In [20]:
import json

import IPython.display
import ipywidgets as widgets
import pandas
import sqlalchemy

import server.model.connection as sm_connection
import server.model.event as sm_event
import server.model.firstapi as sm_firstapi
import server.season.s2019 as s2019
import server.view.excel as sv_excel
import server.view.dataframes as sv_dataframes

In [21]:
def get_measures(match="001-q", phase='auto'):
    conn = sm_connection.engine.connect()
    event_id = sm_event.EventDal.get_current_event()[0]
    sql = sqlalchemy.text("SELECT events.name, matches.name AS match, alliances.name AS alliance, "
                          "stations.name AS station, teams.name AS team, phases.name AS phase, "
                          "tasks.name AS task, measures.attempts, measures.successes, "
                          "measures.cycle_times, measures.capability, task_options.option_name "
                          "FROM measures "
                          "INNER JOIN tasks ON measures.task_id = tasks.id "
                          "INNER JOIN teams ON measures.team_id = teams.id "
                          "INNER JOIN matches ON measures.match_id = matches.id "
                          "INNER JOIN phases ON measures.phase_id = phases.id "
                          "INNER JOIN alliances ON measures.alliance_id = alliances.id "
                          "INNER JOIN stations ON measures.station_id = stations.id "
                          "INNER JOIN events ON measures.event_id = events.id "
                          "LEFT JOIN task_options ON measures.capability = task_options.id "
                          "WHERE event_id = :evt_id AND matches.name = :mtch AND phases.name = :phase "
                          "ORDER BY alliances.name, stations.name, phases.name, tasks.name"
                          ";").bindparams(evt_id=event_id, mtch=match, phase=phase)
    measures_df = pandas.read_sql(sql, conn)
    conn.close()
    return measures_df


def run_cell(cell):
    jscript_cmd = "Jupyter.notebook.execute_cells([" + str(cell) + "])"
    IPython.display.display_javascript(IPython.display.Javascript(jscript_cmd))

    
match = "001-q"
df = None
match_txt = widgets.Text(description="Match", value=match)
match_button = widgets.Button(description="Show Measures")
IPython.display.display(widgets.HBox([match_txt, match_button]))

phase = 'auto'
phase_box = widgets.Dropdown(options=['auto','teleop','finish'],value=phase,description='Phase:',disabled=False,)
phase_button = widgets.Button(description="Set Phase")
IPython.display.display(widgets.HBox([phase_box, phase_button]))


def show_measures(b):
    # IPython.display.clear_output()
    measures_output_cell = 3
    global match
    match = match_txt.value
    global phase 
    phase = phase_box.value
    global df
    df = get_measures(match, phase)
    run_cell(measures_output_cell)
    
def set_phase(b):
    phase = phase_box.value

match_button.on_click(show_measures)
phase_button.on_click(set_phase)

A Jupyter Widget

A Jupyter Widget

In [24]:
df

,name,match,alliance,station,team,phase,task,attempts,successes,cycle_times,capability,option_name


In [23]:
df.query("phase=='teleop'")

,name,match,alliance,station,team,phase,task,attempts,successes,cycle_times,capability,option_name
